In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,DataLoader
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,f1_score

In [2]:
df = pd.read_csv('../input/textdb3/fake_or_real_news.csv')

NameError: name 'pd' is not defined

In [31]:
replacement = {'REAL':0,"FAKE":1}
df['label'].replace(replacement,inplace=True)
df.head()

In [3]:
df['label'].value_counts()

NameError: name 'df' is not defined

In [33]:
df['combined'] = df['title'] + df['text']

In [34]:
X_train,X_test,y_train,y_test = train_test_split(df['combined'],df['label'],random_state=33,test_size=0.3,shuffle=True)

In [35]:
tf = TfidfVectorizer(stop_words=['english'])
train_vectorizer = tf.fit_transform(X_train)
test_vectors = tf.transform(X_test)

In [36]:
X_train = torch.tensor(train_vectorizer.toarray(),dtype=torch.float32)
Y_train = torch.tensor(y_train.values,dtype=torch.float32)
X_test = torch.tensor(test_vectors.toarray(),dtype=torch.float32)
Y_test = torch.tensor(y_test.values,dtype=torch.float32)

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [38]:
model = nn.Sequential( 
    nn.Linear(X_train.shape[1],128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,10),
    nn.ReLU(),
    nn.Linear(10,1),
    nn.Sigmoid()
)
model.to(device)

In [39]:
train_ds  = TensorDataset(X_train,Y_train)
train_dl = DataLoader(train_ds,batch_size=64,shuffle=True)
test_ds = TensorDataset(X_test,Y_test)
test_dl = DataLoader(test_ds,batch_size=64,shuffle=True)

In [40]:
criterion = torch.nn.BCELoss()

In [41]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

**Training Model**

In [42]:
num_epochs = 5
losses,accuracies = [],[]
for epoch in range(num_epochs):
    for x,y in train_dl:
        x,y = x.to(device),y.to(device)
        outputs = model(x)
        loss = criterion(outputs,y.unsqueeze(1))
        accuracy= (model(X_test.to(device)).cpu().reshape(-1).detach().numpy().round() == Y_test.to(device).cpu().detach().numpy()).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    accuracies.append(accuracy)
    print(f"Epoch {epoch} Loss: {loss.item()} Accuracy: {accuracy}")

In [43]:
torch.save(model.state_dict(),"model.pth")

In [44]:
model.load_state_dict(torch.load('model.pth'))

**Testing Model**

In [45]:
torch.round(model(torch.tensor(tf.transform(["Donald Trump’s Shockingly Weak Delegate Game Somehow Got Even Worse"]).toarray(),dtype=torch.float32,device=device)).detach())

**Graph**

In [46]:
print(losses)
plt.plot(losses)
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('loss')

In [47]:
plt.plot(accuracies)
plt.title('Accuracy vs Epochs')
plt.xlabel('Accuracy')
plt.ylabel('loss')

**Confusion Matrix**

In [48]:
cm = confusion_matrix(model(X_test.to(device)).cpu().detach().numpy().round(),Y_test.numpy())
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Real','Fake'])
disp.plot()

**F1 Score**

In [49]:
f1_score(model(X_test.to(device)).cpu().detach().numpy().round(),Y_test.numpy())